In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("ETHNEWTEST.csv")

fd = df[df['Usable'] == True].reset_index(drop=True)
for i in range(len(fd)):
    if i != len(fd) - 1:
        fd.loc[i, 'NTC'] = fd.loc[i + 1, 'Close']
    else:
        fd.loc[i, 'NTC'] = 0

fd[['Close', 'NTC']]

fd = fd.drop(fd.index[-1])
fd = fd.drop('Usable', axis=1)
fd = fd.drop('Date', axis=1)

results_df = pd.DataFrame(columns=['Iteration', 'TotalWin', 'TotalWinW'])

for k in range(200):
    totalwin = 0
    totalwinw = 0
    for m in range(5):
        
        x = fd.drop(['INC', 'NTC'], axis=1)[-(60 + m + k):-(15 + m + k)]
        y = fd[['INC', 'NTC']][-(60 + m + k):-(15 + m + k)]
        x2 = fd.drop(['INC', 'NTC'], axis=1)[-(15 + m + k):] if k == 0 else fd.drop(['INC', 'NTC'], axis=1)[-(15 + m + k):-k]
        y2 = fd[['INC', 'NTC']][-(15 + m + k):] if k == 0 else fd[['INC', 'NTC']][-(15 + m + k):-k]
        
        
        curwin = 0
        curwinw = 0
        for j in range(8):
            
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=j+1)
            y1_train = y_train['INC']
            y2_train = y_train['NTC']
            y1_test = y_test['INC']
            y2_test = y_test['NTC']
            y3_train = y2_train.copy()
            y3_test = y2_test.copy()
            
            model1 = LogisticRegression(solver='liblinear')
            
            model1.fit(x_train, y1_train)
            
            y1_pred = model1.predict(x_test)
            
            model2 = LinearRegression()
            
            model2.fit(x_train, y2_train)
            
            y2_pred = model2.predict(x_test)
            
            xgb_model = xgb.XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.1)
            
            # Fit the model on the training data
            xgb_model.fit(x_train, y3_train)  # y2_train should be continuous values
            
            # Make predictions on the test set
            y3_pred = xgb_model.predict(x_test)
            
            compare = x_test
            
            compare['NTC'] = y2_test
            compare['PNTC'] = y2_pred
            compare['INCT'] = y1_test
            compare['INCP'] = y1_pred
            compare['PNTC2'] = y3_pred
            
            compare['acc'] = ((compare['NTC'] > compare['Close']) & (compare['PNTC'] > compare['Close'])) | \
                             ((compare['NTC'] < compare['Close']) & (compare['PNTC'] < compare['Close']))
            compare['acc'] = compare['acc'].astype(int)
            
            compare['acc3'] = ((compare['NTC'] > compare['Close']) & (compare['PNTC2'] > compare['Close'])) | \
                             ((compare['NTC'] < compare['Close']) & (compare['PNTC2'] < compare['Close']))
            compare['acc3'] = compare['acc3'].astype(int)
            
            compare['acc2'] = compare['INCT'] == compare['INCP']
            
            compare['acctotal'] = compare.apply(lambda row: 
                                                7 if row['acc'] == 1 and row['acc2'] == 1 and row['acc3'] == 1
                                                else 6 if row['acc'] == 1 and row['acc2'] == 1 and row['acc3'] == 0
                                                else 5 if row['acc'] == 1 and row['acc2'] == 0 and row['acc3'] == 1
                                                else 4 if row['acc'] == 1 and row['acc2'] == 0 and row['acc3'] == 0
                                                else 3 if row['acc'] == 0 and row['acc2'] == 1 and row['acc3'] == 1
                                                else 2 if row['acc'] == 0 and row['acc2'] == 1 and row['acc3'] == 0
                                                else 1 if row['acc'] == 0 and row['acc2'] == 0 and row['acc3'] == 1
                                                else 0, axis=1)
            
            
            
            y1_pred_x2 = model1.predict(x2)  # Logistic Regression predictions for INC
            y2_pred_x2 = model2.predict(x2)  # Linear Regression predictions for NTP
            y3_pred_x2 = xgb_model.predict(x2)  # XGBoost predictions for NTP
            
            compare_x2 = x2.copy()
            
            compare_x2['NTC'] = y2['NTC']  # Actual NTP values from y2
            compare_x2['PNTC'] = y2_pred_x2  # Linear Regression predicted NTP values
            compare_x2['INCT'] = y2['INC']  # Actual INC values from y2
            compare_x2['INCP'] = y1_pred_x2  # Logistic Regression predicted INC values
            compare_x2['PNTC2'] = y3_pred_x2  # XGBoost predicted NTP values
            
            # Apply the same accuracy calculations
            compare_x2['acc'] = ((compare_x2['NTC'] > compare_x2['Close']) & (compare_x2['PNTC'] > compare_x2['Close'])) | \
                                ((compare_x2['NTC'] < compare_x2['Close']) & (compare_x2['PNTC'] < compare_x2['Close']))
            compare_x2['acc'] = compare_x2['acc'].astype(int)
            
            compare_x2['acc3'] = ((compare_x2['NTC'] > compare_x2['Close']) & (compare_x2['PNTC2'] > compare_x2['Close'])) | \
                                 ((compare_x2['NTC'] < compare_x2['Close']) & (compare_x2['PNTC2'] < compare_x2['Close']))
            compare_x2['acc3'] = compare_x2['acc3'].astype(int)
            
            compare_x2['acc2'] = compare_x2['INCT'] == compare_x2['INCP']
            
            # Apply the same 'acctotal' logic
            compare_x2['acctotal'] = compare_x2.apply(lambda row: 
                                                    7 if row['acc'] == 1 and row['acc2'] == 1 and row['acc3'] == 1
                                                    else 6 if row['acc'] == 1 and row['acc2'] == 1 and row['acc3'] == 0
                                                    else 5 if row['acc'] == 1 and row['acc2'] == 0 and row['acc3'] == 1
                                                    else 4 if row['acc'] == 1 and row['acc2'] == 0 and row['acc3'] == 0
                                                    else 3 if row['acc'] == 0 and row['acc2'] == 1 and row['acc3'] == 1
                                                    else 2 if row['acc'] == 0 and row['acc2'] == 1 and row['acc3'] == 0
                                                    else 1 if row['acc'] == 0 and row['acc2'] == 0 and row['acc3'] == 1
                                                    else 0, axis=1)
            
            # Display the value counts for 'acctotal' on x2
            compare_x2['acctotal'].value_counts()
            
            weightcompare = pd.DataFrame([0, 0, 0, 0, 0, 0, 0, 0], index=[0, 1, 2, 3, 4, 5, 6, 7], columns=['Value'])
            
            for i in range(len(compare_x2) - 1):
                weightcompare.iloc[compare_x2.iloc[i]['acctotal']] += (i + 1)
                
                  
            last_value = compare_x2['acctotal'].iloc[-1]
            
            
            win = 0
            if compare_x2['acctotal'].value_counts().get(last_value, 0) - 1 > compare_x2['acctotal'].value_counts().get(7 - last_value, 0):
                win = 1
            elif compare_x2['acctotal'].value_counts().get(last_value, 0) - 1 == compare_x2['acctotal'].value_counts().get(7 - last_value, 0):
                win = 0
            else:
                win = -1
            
            
            v1 = weightcompare['Value'].iloc[last_value]
            v2 = weightcompare['Value'].iloc[7 - last_value]
            winw = 0
            if v1 > v2:
                winw = 1
            elif v1 == v2:
                winw = 0
            else:
                winw = -1
            
            curwin += win
            curwinw += winw
            
        if curwin > 0:
            totalwin += 1
        elif curwin < 0:
            totalwin -= 1
    
        if curwinw > 0:
            totalwinw += 1
        elif curwinw < 0:
            totalwinw -=1
            
        new_row = pd.DataFrame({'Iteration': [k], 'TotalWin': [totalwin], 'TotalWinW': [totalwinw]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)
            
    

/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda3/envs/env/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: Conv

In [190]:
"""
y1_pred_x2 = model1.predict(x2)  # Logistic Regression predictions for INC
y2_pred_x2 = model2.predict(x2)  # Linear Regression predictions for NTP
y3_pred_x2 = xgb_model.predict(x2)  # XGBoost predictions for NTP

compare_x2 = x2.copy()

compare_x2['NTC'] = y2['NTC']  # Actual NTP values from y2
compare_x2['PNTC'] = y2_pred_x2  # Linear Regression predicted NTP values
compare_x2['INCT'] = y2['INC']  # Actual INC values from y2
compare_x2['INCP'] = y1_pred_x2  # Logistic Regression predicted INC values
compare_x2['PNTC2'] = y3_pred_x2  # XGBoost predicted NTP values

# Apply the same accuracy calculations
compare_x2['acc'] = ((compare_x2['NTC'] > compare_x2['Close']) & (compare_x2['PNTC'] > compare_x2['Close'])) | \
                    ((compare_x2['NTC'] < compare_x2['Close']) & (compare_x2['PNTC'] < compare_x2['Close']))
compare_x2['acc'] = compare_x2['acc'].astype(int)

compare_x2['acc3'] = ((compare_x2['NTC'] > compare_x2['Close']) & (compare_x2['PNTC2'] > compare_x2['Close'])) | \
                     ((compare_x2['NTC'] < compare_x2['Close']) & (compare_x2['PNTC2'] < compare_x2['Close']))
compare_x2['acc3'] = compare_x2['acc3'].astype(int)

compare_x2['acc2'] = compare_x2['INCT'] == compare_x2['INCP']

# Apply the same 'acctotal' logic
compare_x2['acctotal'] = compare_x2.apply(lambda row: 
                                        7 if row['acc'] == 1 and row['acc2'] == 1 and row['acc3'] == 1
                                        else 6 if row['acc'] == 1 and row['acc2'] == 1 and row['acc3'] == 0
                                        else 5 if row['acc'] == 1 and row['acc2'] == 0 and row['acc3'] == 1
                                        else 4 if row['acc'] == 1 and row['acc2'] == 0 and row['acc3'] == 0
                                        else 3 if row['acc'] == 0 and row['acc2'] == 1 and row['acc3'] == 1
                                        else 2 if row['acc'] == 0 and row['acc2'] == 1 and row['acc3'] == 0
                                        else 1 if row['acc'] == 0 and row['acc2'] == 0 and row['acc3'] == 1
                                        else 0, axis=1)

# Display the value counts for 'acctotal' on x2
compare_x2['acctotal'].value_counts()
"""

"\ny1_pred_x2 = model1.predict(x2)  # Logistic Regression predictions for INC\ny2_pred_x2 = model2.predict(x2)  # Linear Regression predictions for NTP\ny3_pred_x2 = xgb_model.predict(x2)  # XGBoost predictions for NTP\n\ncompare_x2 = x2.copy()\n\ncompare_x2['NTC'] = y2['NTC']  # Actual NTP values from y2\ncompare_x2['PNTC'] = y2_pred_x2  # Linear Regression predicted NTP values\ncompare_x2['INCT'] = y2['INC']  # Actual INC values from y2\ncompare_x2['INCP'] = y1_pred_x2  # Logistic Regression predicted INC values\ncompare_x2['PNTC2'] = y3_pred_x2  # XGBoost predicted NTP values\n\n# Apply the same accuracy calculations\ncompare_x2['acc'] = ((compare_x2['NTC'] > compare_x2['Close']) & (compare_x2['PNTC'] > compare_x2['Close'])) |                     ((compare_x2['NTC'] < compare_x2['Close']) & (compare_x2['PNTC'] < compare_x2['Close']))\ncompare_x2['acc'] = compare_x2['acc'].astype(int)\n\ncompare_x2['acc3'] = ((compare_x2['NTC'] > compare_x2['Close']) & (compare_x2['PNTC2'] > compare_

In [175]:
weightcompare = pd.DataFrame([0, 0, 0, 0, 0, 0, 0, 0], index=[0, 1, 2, 3, 4, 5, 6, 7], columns=['Value'])
for i in range(len(compare_x2) - 1):
    weightcompare.iloc[compare_x2.iloc[i]['acctotal']] += (i + 1)

In [176]:
weightcompare, compare_x2[-1:]['acctotal']

(   Value
 0     10
 1     37
 2      0
 3     37
 4     28
 5     23
 6     75
 7      0,
 393    1
 Name: acctotal, dtype: int64)

In [189]:
"""
last_value = compare_x2['acctotal'].iloc[-1]
win = 0
if compare_x2['acctotal'].value_counts().get(last_value, 0) - 1 > compare_x2['acctotal'].value_counts().get(7 - last_value, 0):
    win = 1
elif compare_x2['acctotal'].value_counts().get(last_value, 0) - 1 == compare_x2['acctotal'].value_counts().get(7 - last_value, 0):
    win = 0
else:
    win = -1


v1 = weightcompare['Value'].iloc[last_value]
v2 = weightcompare['Value'].iloc[7 - last_value]

winw = 0
if v1 > v2:
    winw = 1
elif v1 == v2:
    winw = 0
else:
    winw = -1

win, winw
"""

(-1, -1)